In [1]:
import numpy as np
import sys, os
sys.path.append('../../shared/utilsGRF/')
from IPython.display import display
import networkx as nx
import subprocess, glob, re
import matplotlib.pyplot as plt

In [2]:
edges=[(1,'a1',2),(2,'a1r',1),(2,'a2',3),(3,'a2r',2),(3,'a3',4),(4,'a3r',3),(4,'a4',1),(1,'a4r',4),
       (5,'alpha1',6),(6,'alpha1r',5),(6,'alpha2',7),(7,'alpha2r',6),(7,'alpha3',8),(8,'alpha3r',7),(8,'alpha4',5),(5,'alpha4r',8),
      (1,'kb1-x',5),(5,'ku1',1),(2,'kb2-x',6),(6,'ku2',2),(3,'kb3-x',7),(7,'ku3',3),(4,'kb4-x',8),(8,'ku4',4)]

nnodes=8
MTTfolder='../../shared/utilsGRF' #folder with MTT.py 
basename='graph'

parlist=[x[1] for x in edges]
parlist=list(set(parlist))


for pnum,par in enumerate(parlist):
    if '-x' in par:
        parlist[pnum]=parlist[pnum].replace('-x','')
print(parlist)

graphname=os.path.join(MTTfolder,basename+'.txt')
outf=open(graphname,'w')
for x in edges:
    x_str=list(map(str,x))
    outf.write('('+','.join(x_str)+') ')
outf.close()

#execute_MTT(self):

args=["python",os.path.join(MTTfolder,'MTT.py'),graphname]
print("executing MTT")
print(subprocess.check_call(args))


#parse_rhos_from_graphfiles(self):
#this was previously called rhos_from_edges
edges=np.array(edges)
#find files that correspond to the spanning trees
files=glob.glob(os.path.join(MTTfolder,basename+'*'))
pat=re.compile('[0-9]+.txt')
n=0
for f in files:
    if pat.findall(f):
        n+=1
all_rhos=[] 
allGraphs=dict()
allterms=dict()
for i in range(1,n+1):
    fname=os.path.join(MTTfolder,basename+'-%d.txt'%i)

    fi=open(fname,'r')
    rho_expr='rho_%d='%i
    allGraphs[i]=[]
    allterms[i]=[]
    for lnum,l in enumerate(fi.readlines()):
        idxs=np.array(list(map(int,l.strip().split())),dtype=bool)
        subset=edges[idxs]
        expr=[x[1] for x in subset] #labels that multiply together in a term of the rho
        term='*'.join(expr)
        if lnum<1:
            rho_expr=rho_expr+term
        else:
            rho_expr=rho_expr+'+'+term
        allterms[i].append(term)
        #now get Graph
        G=nx.DiGraph()
        for edge in subset:
            n0=int(edge[0])
            n1=int(edge[2])
            par=edge[1]
            G.add_edge(n0,n1,name=par)
            G.add_edge
        allGraphs[i].append(G)
        
            
        

    all_rhos.append(rho_expr)

['ku2', 'a4r', 'alpha3', 'kb1', 'alpha2', 'a1', 'ku3', 'a3', 'alpha4r', 'a3r', 'ku4', 'alpha1r', 'alpha3r', 'kb2', 'a2r', 'alpha2r', 'a1r', 'kb3', 'ku1', 'alpha1', 'a2', 'a4', 'kb4', 'alpha4']
executing MTT
0


In [3]:
import re
kbpat=re.compile("kb")
alphapat=re.compile("alpha[0-9][^r]")
alpharepat=re.compile("alpha[0-9]r")

In [4]:
set_1=set(["1,2","3,4","5,6"])
set_2=set(["1,2","3,4","50,60"])
set_1.symmetric_difference(set_2)

{'5,6', '50,60'}

#Get mapping between trees rooted at bound state and trees rooted at unbound state:
- if there is a binding edge to the root: change for the unbinding edge and that's it
- if there is not a binding edge to the root: reverse minimal path. This should result in less alphas in the tree rooted at the unbound


In [5]:
wholegraph=nx.DiGraph()
for edge in edges:
    wholegraph.add_edge(int(edge[0]),int(edge[2]))

In [ ]:
#the root+term number is an id for the trees

In [25]:

nnodes=4
correspondance=[]
for node in range(nnodes+1,nnodes+1+nnodes):
    
    for n in range(1,nnodes+1): #for trees with n kb edges
        
        #Gs_b_kb=[] #trees with kb directly to root node
        #Gs_b_nokb=[] #trees with no kb directly to root node
        Gs_b=[]
        IDs=[]
        termsb=[]
        for t,term in enumerate(allterms[node]):
            kbs=kbpat.findall(term)
            if len(kbs)==n:
                G=allGraphs[node][t]
                Gs_b.append(G)
                IDs.append(t)
                termsb.append(term)
                #if "kb%d-x"%node in term:
                #    Gs_b_kb.append(G)
                #else:
                #    Gs_b_nokb.append(G)
        
        #now check that for each of those trees there is a corresponding tree rooted at an unbound node
        
        
        for g,G in enumerate(Gs_b):
            
            
            node_u=node-nnodes
            #if True:
            for node_u in range(1,nnodes+1):
                #look if I have a tree rooted at the unbound that correspond to this graph reversing the minimal path.
                #reversing one binding edge only
                for t,term in enumerate(allterms[node_u]): 
                    kbs=kbpat.findall(term)
                    if len(kbs)==(n-1): #
                    #if True:
                        Gu=allGraphs[node_u][t]
                        mp=nx.shortest_path(Gu,source=node,target=node_u) #since the tree GU is spanning rooted at node_u, for sure there is a path
                        edges_mp=[[mp[i],mp[i+1]] for i in range(len(mp)-1)]

                        #get presumably corresponding tree rooted at unbound from the tree rooted at bound
                        edgesu=[]
                        for edge in G.edges():
                            edge_r=[edge[1],edge[0]]
                            if not edge_r in edges_mp:
                                edgesu.append(",".join(map(str,edge)))
                            else:
                                edgesu.append(",".join(map(str,edge_r)))
                        edgesu=set(edgesu)
                        newedgesu=[",".join(map(str,x)) for x in Gu.edges()]

                        newedgesu=set(newedgesu)
                        if len(edgesu.symmetric_difference(newedgesu))==0:
                            nalphasb=len(alphapat.findall(termsb[g]))
                            nalphasu=len(alphapat.findall(term))
                            if nalphasb-nalphasu>=0:
                                correspondance.append([node,n,IDs[g],node_u,t,nalphasb-nalphasu])
                            #print("Ge",G.edges())
                            #print("Gue",Gu.edges())
                        
                    
                    
            
                
                        

In [8]:
import pandas as pd

In [26]:
df=pd.DataFrame(np.vstack(correspondance),columns=["node","nkb","IDb","node_u","IDu","alphadif"])
df["unique_u"]=[x[0]+"-"+x[1] for x in list(zip(df["node_u"].astype(str),df["IDu"].astype(str)))]

In [27]:
[len(allterms[x]) for x in allterms]

[384, 384, 384, 384, 384, 384, 384, 384]

In [28]:
ntree=384
for nodeb in range(5,5+nnodes):
    subdf=df[df["node"]==nodeb]
    ar=subdf["IDb"].values
    for i in range(ntree):
        if not i in ar:
            print("not ",i)

not  163
not  164
not  167
not  170
not  174
not  175
not  178
not  181
not  198
not  199
not  202
not  205
not  209
not  210
not  212
not  213
not  216
not  217
not  221
not  222
not  224
not  225
not  226
not  228
not  229
not  233
not  235
not  237
not  240
not  266
not  267
not  269
not  270
not  271
not  273
not  274
not  278
not  280
not  282
not  285
not  290
not  291
not  293
not  294
not  295
not  297
not  298
not  301
not  302
not  304
not  305
not  306
not  307
not  310
not  311
not  313
not  314
not  315
not  319
not  321
not  323
not  326
not  338
not  339
not  347
not  350
not  379
not  382
not  36
not  41
not  46
not  48
not  49
not  54
not  56
not  57
not  62
not  64
not  65
not  110
not  115
not  117
not  118
not  224
not  229
not  231
not  232
not  235
not  236
not  239
not  242
not  246
not  248
not  251
not  252
not  253
not  256
not  257
not  258
not  260
not  261
not  264
not  265
not  266
not  268
not  269
not  272
not  273
not  274
not  276
not  277
not  286
not

In [21]:

for nodeu in range(1,1+nnodes):
    subdf=df[df["node_u"]==node_u]
    ar=subdf["IDu"].values
    for i in range(ntree):
        if not i in ar:
            print("not ",i)

In [22]:
ids,counts=np.unique(df["unique_u"],return_counts=True)
min(counts),max(counts)

(1, 1)

In [24]:
min(df["alphadif"])

-3